In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multi-lingual-sentiment-analysis/sample_submission.csv
/kaggle/input/multi-lingual-sentiment-analysis/train.csv
/kaggle/input/multi-lingual-sentiment-analysis/test.csv
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/

In [2]:
#%%capture
!pip install bitsandbytes
!pip install accelerate
!pip install peft
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 65.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0


In [3]:
from datetime import datetime
import torch
import pandas as pd
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

In [4]:
import torch
model_path = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"


# Quantization configuration

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Loading the model and tokenizer

modelCLM = AutoModelForCausalLM.from_pretrained(model_path,quantization_config=bnb_config,
                                             device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=1200,
    padding_side="left",
    add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
pip show transformers

Name: transformers
Version: 4.48.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: kaggle-environments, peft, sentence-transformers
Note: you may need to restart the kernel to use updated packages.


from transformers import AutoModelForSequenceClassification

modelCLS = AutoModelForSequenceClassification.from_pretrained(model_path,quantization_config=bnb_config,
                                             device_map="auto")

from transformers import TextClassificationPipeline
classifier = TextClassificationPipeline(model=modelCLS,
                                       tokenizer=tokenizer,
                                       framework='pt',
                                       task="sentiment-analysis",
                                       #device = "cuda"
                                       )

In [6]:
import pandas as pd
from datasets import load_dataset, Dataset

In [7]:
file_path = "/kaggle/input/multi-lingual-sentiment-analysis/test.csv"

df = pd.read_csv(file_path,index_col=0)
df = pd.DataFrame(df)

test_dataset = Dataset.from_pandas(df)
print(f"Test set shape: {test_dataset.shape}")

Test set shape: (100, 3)


In [8]:
file_path = "/kaggle/input/multi-lingual-sentiment-analysis/train.csv"

df = pd.read_csv(file_path,index_col=0)
df = pd.DataFrame(df)

train_dataset = Dataset.from_pandas(df)
print(f"Training set shape: {train_dataset.shape}")

Training set shape: (1000, 4)


In [9]:
#sub_file_path = "/kaggle/input/multi-lingual-sentiment-analysis/sample_submission.csv"

#dfSub = pd.read_csv(sub_file_path,index_col=0)
#dfSub = pd.DataFrame(dfSub)

#dfSub_dataset = Dataset.from_pandas(dfSub)
#print(f"Submission set shape: {dfSub_dataset.shape}")


In [10]:
#dfSub

In [11]:
L_Positive = "Positive"
L_Negative = "Negative"

id2label = {0: L_Positive, 1: L_Negative }
label_mapping = {
    "LABEL_0": L_Positive,
    "LABEL_1": L_Negative
}

In [12]:
def SACLS_Prompt(input):
#    prompt = f'''Find sentiment of this {map_dict[input['language']]} Text: {input['sentence']}'''
#    return prompt
    return input
    
def BATCH_SACLS_Sentiment(dataset, classifier):
    predList=[]
    for i in range(dataCount):
        prompt = SACLS_Prompt(dataset['sentence'][i])
        predLabel = label_mapping.get(classifier(prompt)[0]['label'])
        predList.append(predLabel)

    return predList

In [13]:
def CLS_Sentiment(prompt, model, tokenizer,max_length=1200):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model(**inputs)
    id=torch.argmax(outputs.logits, 1)
    label=id2label[int(id)]
    score=torch.softmax(outputs.logits, 1)
    score = float(max(score[0]))
    return label

In [14]:
lang_dict = {
    "as": "Assamese",
    "bd": "Bodo",
    "bn": "Bengali",
    "gu": "Gujarati",
    "hi": "Hindi",
    "kn": "Kannada",
    "ml": "Malayalam",
    "mr": "Marathi",
    "or": "Odia",
    "pa": "Punjabi",
    "ta": "Tamil",
    "te": "Telugu",
    "ur": "Urdu"
}


dataCount=1000
count=0
for i in range(dataCount):
    text = train_dataset['sentence'][i]
    refLabel = train_dataset['label'][i]
    predLabel = label_mapping.get(classifier(text)[0]['label'])
    if refLabel==predLabel:
        count = count+1
    if i%5==0:
        print(i, count/(i+1))

print(i, count/dataCount)

In [15]:
#    Determine if it is positive or negative, and return the answer as the corresponding sentiment label "Positive" or  "Negative".
#    Give only one output. Stop after writing the predicted sentiment.
#     Stop once you write '_SENTIMENT_ = Positive' or '_SENTIMENT_ = Negative'
#    Give the answer in this format: '_SENTIMENT_ = <predicted-sentiment>'

SENTIMENT_SEP = '_SENTIMENT_'

def CLM_Prompt(input):
    prompt1 = f'''
    Analyse the sentiment of this {lang_dict[input['language']]} sentence enclosed in square brackets.
    Determine if it is positive or negative, and return the answer as the corresponding sentiment label.
    Do not use any programming method and use your domain knowledge.
    No explanation. Do not provide any additional note.
    
    [ {input['sentence']} ]

    label: 
    '''

    prompt2 = f'''Predict the sentiment of the following sentence: {input['sentence']} as either positive or negative
    
    This sentence comes from the {lang_dict[input['language']]} language.
    Please provide your answer  as per the given structure: `The predicted sentiment of the statement is <predicted-sentiment>.`
    Try to avoid any programming method and use your domain knowledge
    Don't generate any explanation or reasoning in support of your answer'''

    prompt3=f'''
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful assistant. Predict the sentiment of the given {lang_dict[input['language']]} language sentence as POSITIVE or NEGATIVE. <|eot_id|>
    <|start_header_id|>user<|end_header_id|>
    {input['sentence']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
    '''

    prompt4=f'''
    Analyze the sentiment of the following text. Classify the sentiment as either "Positive" or "Negative" only. Do not classify it as Neutral or give any other responses. Return just the sentiment without any additional explanation. Do not use any programming language.

    If you cannot determine the sentiment, translate the text to English and classify the sentiment again. Return just the sentiment without any additional explanation.

    Text: "{input['sentence']}"

    Sentiment:
    '''

    prompt5=f'''
    Analyse the sentiment of  this {lang_dict[input['language']]} text and Classify it into positive or negative, and return the answer as the corresponding sentiment label.
    text: {input["sentence"]}
    label: 
    '''
    return prompt1

In [16]:
def CLM_Sentiment(prompt, model, tokenizer, max_length=200):
    # Tokenize input prompt and move to the correct device
    tokens = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate text
    with torch.no_grad():
        pred = model.generate(**tokens, 
                              max_new_tokens=2,
                              temperature=0.1)

    # Decode and return generated text
    return tokenizer.decode(pred[0], skip_special_tokens=True)

In [17]:
loPositive="positive"
loNegative="negative"

In [18]:
def PostProcess(text):
    text = text.lower()
    #index = output.find(SENTIMENT_SEP)
    countPositive = text.count(loPositive)
    countNegative = text.count(loNegative)

    if countPositive > countNegative:
        return L_Positive

    return L_Negative

In [19]:
def BATCH_SentimentCLM(model,tokenizer, dataset):
    size = len(dataset)
    predList = []
    errCount=0
    for i in dataset:
        prompt = CLM_Prompt(i)
        output = CLM_Sentiment(prompt, model, tokenizer,max_length=1200)
#        print("Base=", i['label'])
#        print(output)
#        index = output.find(SENTIMENT_SEP)
        predLabel = PostProcess(output)
        """
        if index!=-1:
            iPositive = output[index:].lower().find(loPositive)
            iNegative = output[index:].lower().find(loNegative)
            if iPositive>0:
                predLabel = L_Positive
            elif iNegative>0:
                predLabel = L_Negative
            
        if predLabel is None:
            errCount = errCount+1
            print('NOT_FOUND:', errCount) 
            predLabel = L_Positive
        """    
        print("Out=",predLabel)
        predList.append(predLabel)
    return predList

In [20]:
predList = None
predList = BATCH_SentimentCLM(modelCLM,tokenizer, test_dataset)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Out= Positive


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Out= Negative


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Out= Positive


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Out= Negative


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Out= Positive
Out= Negative


In [21]:
import pandas as pd
def MakeSubmission(predList):

    if predList is None:
        predList = ['Positive'] * 100  # Set all labels to "Positive"
    # Create a DataFrame with IDs from 1 to 100 and label "Positive"
    data = {
        'ID': range(1, len(predList)+1),  # Generate IDs from 1 to 100
#        'label': ['Positive'] * 100  # Set all labels to "Positive"
        'label': predList
    }

    # Create the DataFrame
    submission_df = pd.DataFrame(data)
    
    # Save it to a CSV file
    submission_df.to_csv("submission.csv", index=False)

    return submission_df

In [22]:
submission_df=MakeSubmission(predList)
print(submission_df)

     ID     label
0     1  Positive
1     2  Negative
2     3  Positive
3     4  Positive
4     5  Negative
..  ...       ...
95   96  Negative
96   97  Positive
97   98  Negative
98   99  Positive
99  100  Negative

[100 rows x 2 columns]


In [23]:
#df_Out = pd.DataFrame(predList,columns=['label'])
#df_Out.to_csv('submission.csv')